# Extract blocks of trials from the whole runs

In [15]:
import os
from os.path import join as opj
import numpy as np
import nibabel as nib
import pandas as pd
import re

In [16]:
SET = 'XP1'
SUBSET = ''
dir_to_concat = opj(SET, SUBSET) if SUBSET != '' else SET
INPUT_DIR = opj(r"...\data\ds002338\PreProcessed\PreProcessed", dir_to_concat)  # need to be set
OUTPUT_DIR = opj(r"...\data\ds002338\PreProcessed\Trials", dir_to_concat)  # need to be set
LABEL_CSV = opj(r"...\data\ds002338\PreProcessed\Trials", dir_to_concat, "labels.csv")  # need to be set
TASK = 'eegfmriNF'

In [17]:
event_file = opj(r"...\data\ds002338\PreProcessed\PreProcessed", SET, f"task-{TASK}_events.tsv") if SUBSET == '' else \
             opj(r"...\data\ds002338\PreProcessed\PreProcessed", SET, SUBSET, f"task-{TASK}_events.tsv")    # need to be set

trialinfo = pd.read_table(event_file)
trialinfo.head()

if SET == 'XP1':
    trialinfo['onset'] = trialinfo['onset'] // 2
    trialinfo['duration'] = trialinfo['duration'] // 2

trial_type_ids = {
    'Rest': 0,
    'Task-NF': 1,
}

trialinfo

,onset,duration,trial_type,stim_file
0,1,10,Rest,Rest_Xp1.png
1,11,10,Task-NF,Task-NF_Xp1.png
2,21,10,Rest,Rest_Xp1.png
3,31,10,Task-NF,Task-NF_Xp1.png
4,41,10,Rest,Rest_Xp1.png
5,51,10,Task-NF,Task-NF_Xp1.png
6,61,10,Rest,Rest_Xp1.png
7,71,10,Task-NF,Task-NF_Xp1.png
8,81,10,Rest,Rest_Xp1.png
9,91,10,Task-NF,Task-NF_Xp1.png


In [18]:
# Collect per subject, pre run fMRI files
fmri_pths = []

for subj_dir in os.listdir(INPUT_DIR):
    subj_dir_pth = opj(INPUT_DIR, subj_dir, f'task-{TASK}') if SUBSET != '' else opj(INPUT_DIR, subj_dir, f'task-{TASK}')
    if not os.path.isdir(subj_dir_pth):
        continue
    if SET == 'XP2':
        for run_dir in os.listdir(subj_dir_pth):
            run_dir_pth = opj(subj_dir_pth, run_dir)
            for file_name in os.listdir(run_dir_pth):
                if os.path.splitext(file_name)[-1] == '.gz' and 'fwhm' in file_name:
                    fmri_pths.append(
                        opj(run_dir_pth, file_name)
                    )
    else:
        for file_name in os.listdir(subj_dir_pth):
                if os.path.splitext(file_name)[-1] == '.gz' and 'fwhm' in file_name:
                    fmri_pths.append(
                        opj(subj_dir_pth, file_name)
                    )

In [19]:
subjects = []
runs = []
ext_fmri_pths = []
trial_types = []
trial_ids = []

for fmri_pth in fmri_pths:
    fmri_data = nib.load(fmri_pth)

    fmris = 0
    for index, row in trialinfo.iterrows():
        trial_data = fmri_data.slicer[..., row['onset']:row['onset']+row['duration']]
        trial_type = row['trial_type']
        trial_id = trial_type_ids[trial_type]
        subject = re.findall('sub-xp\d\d\d', fmri_pth)[0].split('sub-xp')[-1]
        
        if SET == 'XP2':
            run = re.findall('run-\d\d', fmri_pth)[0].split('run-')[-1]
            subject_run_dir = opj(OUTPUT_DIR, f'sub-{subject}', f'run-{run}')
        else:
            run = '-'
            subject_run_dir = opj(OUTPUT_DIR, f'sub-{subject}')
        
        os.makedirs(subject_run_dir, exist_ok=True)
        
        ext_frmi_pth = opj(subject_run_dir, f'{fmris}_{trial_type}_fmri.nii.gz')
        nib.save(trial_data, ext_frmi_pth)
        
        subjects.append(subject)
        runs.append(run)
        trial_ids.append(trial_id)
        trial_types.append(trial_type)
        ext_fmri_pths.append(ext_frmi_pth)
        
        fmris += 1

label_dict = {
    'subjects': subjects,
    'runs': runs,
    'trial_ids': trial_ids,
    'trial_types': trial_types,
    'ext_frmi_pths': ext_fmri_pths
}
label_df = pd.DataFrame.from_dict(label_dict)
label_df.to_csv(LABEL_CSV, sep=';')